In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import expipe
from expipe import Browser
import pandas as pd
import numpy as np
import sys
import exdir

sys.path.append('../ca2-mec')
import data_processing as dp

In [3]:
project_path = dp.project_path()
#project_path = '../../CA2prosjektmappe'
project = expipe.get_project(project_path)
actions = project.actions

In [4]:
dataframes = actions['dataframes']
sessions = pd.read_csv(dataframes.data_path('sessions'))
sessions.head()
print(len(sessions), len(np.unique(sessions.action)))

5 5


In [5]:
def process(data_loader: dp.Data, y0:float=0.7,x0:float=0.3,y1:float=0.3,x1:float=0.7,
           m_y0:float=0.12,m_x0:float=0.12,m_y1:float=0.12,m_x1:float=0.12, t_start:float=0.0, t_stop:float=None):
    """
    Args:
        y0: top left y-coordinate for social boundary
        x0: top left x-coordinate for social boundary
        y1: bottom right y-coordinate for social boundary
        x1: bottom right x-coordinate for social boundary
        m_y0: top left transparent box social margin y-direction
        m_x0: top left transparent box social margin x-direction
        m_y1: bottom right transparent box social margin y-direction
        m_x1: bottom right transparent box social margin x-direction
        t_start: Choose tracking data START time (IN SECONDS :D)
        t_stop: Choose tracking data END time (IN SECONDS :D)
    """
    def process_row(row) -> None:
        action_id = row['action']
        x, y, t, speed = map(data_loader.tracking(action_id).get, ['x', 'y', 't', 'v']) 
        
        # Choose tracking data time interval (IN SECONDS :D)
        if t_start is not None and t_stop is not None:
            mask = (t < t_stop) & (t > t_start)
            x, y, t = x[mask], y[mask], t[mask]
        
        total_recording_time = t[-1]

        # Create social zone boundaries
        s_y0,s_x0,s_y1,s_x1 = y0-m_y0,x0+m_x0,y1+m_y1,x1-m_x1

        # top left social indecies 
        mask = x < s_x0
        mask *= (y > s_y0)
        # bottom right social indecies
        mask2 = x > s_x1
        mask2 *= (y < s_y1)

        # Can use masks to index the spatial coordinates corresponding to social interactions.
        # This is, however, not a wanted/needed quantity for storage and, therefore, skipped.
        """
        mask += mask2
        social_x = x[mask]
        social_y = y[mask]
        """
        
        # Assuming uniformly spaced recording times and (the time array) t is cumulative, then:
        samp_freq = len(t) / t[-1]
        top_right_time = np.sum(mask) / samp_freq
        bottom_left_time = np.sum(mask2) / samp_freq
        #print(samp_freq, time_spent_in_zone_1, time_spent_in_zone_2, np.sum(mask), np.sum(mask2))
        
        # Social discrimination index between time spent in close proximity to the transparent boxes
        # High positive value (close to 1) indicates preferred socialization with rat in top right corner. 
        # In contrast, high negative value (close to -1) indicates preferred socialization with rat 
        # in bottom left corner.
        sdi = (top_right_time - bottom_left_time) / (top_right_time + bottom_left_time)
        
        # Save values
        row['total-time-in-seconds'] = total_recording_time
        row['time-spent-in-top-right-social-zone'] = top_right_time
        row['time-spent-in-bottom-left-social-zone'] = bottom_left_time
        row['social-discrimination-index'] = sdi
        print(row)
        
    return process_row


In [6]:
max_speed = 1 # m/s only used for speed score
min_speed = 0.02 # m/s only used for speed score
position_sampling_rate = 100 # for interpolation
position_low_pass_frequency = 6 # for low pass filtering of position

box_size = [1.0, 1.0]
bin_size=0.01
#smoothing = 0.05
baseline_duration = None

# Constants describing the transparent boxes and social margin used for annotation of social interactions
# See the function "process" in the cell above for a description.
y0,x0,y1,x1 = 0.68, 0.3, 0.31, 0.68 # manually adjust box-borders
m_y0,m_x0,m_y1,m_x1=0.12, 0.12, 0.12, 0.12
t_start,t_stop = 0.0,300


data_loader = dp.Data(
    position_sampling_rate=position_sampling_rate, 
    position_low_pass_frequency=position_low_pass_frequency,
    box_size=box_size, bin_size=bin_size, stim_mask=False, baseline_duration=baseline_duration, 
)



sessions.apply(process(data_loader,y0,x0,y1,x1,m_y0,m_x0,m_y1,m_x1,t_start,t_stop), axis=1)

action                                   013-050521-1
total-time-in-seconds                         299.997
time-spent-in-top-right-social-zone           61.4334
time-spent-in-bottom-left-social-zone         95.2053
social-discrimination-index                 -0.215604
Name: 0, dtype: object
action                                   013-050521-2
total-time-in-seconds                             300
time-spent-in-top-right-social-zone            30.111
time-spent-in-bottom-left-social-zone         65.6622
social-discrimination-index                 -0.371202
Name: 1, dtype: object
action                                   013-050521-3
total-time-in-seconds                         299.994
time-spent-in-top-right-social-zone           51.6124
time-spent-in-bottom-left-social-zone         66.4631
social-discrimination-index                 -0.125773
Name: 2, dtype: object
action                                   013-050521-4
total-time-in-seconds                         299.992
time-spent-in

0    None
1    None
2    None
3    None
4    None
dtype: object

In [ ]:
sessions.to_csv(dataframes.data_path('sessions'), index=False)
#rediger navn i fileexplorer